## Transfer Learning Applied to Fashion MNIST

In [ ]:
import tensorflow as tf

## Reading Fashion MNIST Data

Download Fashion MNIST Dataset from https://github.com/zalandoresearch/fashion-mnist

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data

#Fashion MNIST dataset is downloaded to data/fashion directory
fashion = input_data.read_data_sets("data/fashion", one_hot=True)

In [ ]:
print(fashion.train.num_examples) # Number of training data
print(fashion.test.num_examples) # Number of test data

## Setting hyper-parameters

In [ ]:
# architecture hyper-parameter
learning_rate = 0.01
training_iters = 100000
batch_size = 128
display_step = 20

n_input = 784 # 28x28 image
n_classes = 10 # 1 for each digit [0-9]
dropout = 0.75 

## Defining x and y as placeholders

In [ ]:
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) #For dropout
print(x.shape, y.shape)

## Defining model using conv2d, Relu and Maxpooling

In [ ]:
#This is convolution layer
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

In [ ]:
#This is maxpool layer
def maxpool2d(x, k=2): #k is the size of maxpool filter
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

In [ ]:
#This is our CNN
def conv_net(x, weights, biases, dropout):
    # reshape input to 28x28 size,
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution layer 1
    conv1 = conv2d(x, weights['wc1'], biases['bc1']) 
    # Max pooling
    conv1 = maxpool2d(conv1, k=2) 

    # Convolution layer 2
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2']) 
    # Max pooling
    conv2 = maxpool2d(conv2, k=2) 

    # Fully connected layer
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]]) 
    
    #After that line, we'll have very similar steps to Multilayer perceptron
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1) 
    fc1 = tf.nn.dropout(fc1, dropout) # dropout

    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

## Initializing Weights and Biases

In [ ]:
wc1 = tf.get_variable('wc1', shape=[5, 5, 1, 32])
wc2 = tf.get_variable('wc2', shape=[5, 5, 32, 64])
wd1 = tf.get_variable('wd1', shape=[7*7*64, 1024])
weight_out = tf.get_variable('weight_out', shape=[1024, n_classes])

weights = {'wc1': wc1, 'wc2': wc2, 'wd1': wd1, 'out': weight_out}


bc1 = tf.get_variable('bc1', shape=[32])
bc2 = tf.get_variable('bc2', shape=[64])
bd1 = tf.get_variable('bd1', shape=[1024])
bias_out = tf.get_variable('bias_out', shape=[n_classes])

biases = {'bc1': bc1, 'bc2': bc2, 'bd1': bd1, 'out': bias_out}

## Define cost, optimizer and accuracy

In [ ]:
# Create the model
model = conv_net(x, weights, biases, keep_prob)
print(model)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=y)) #Softmax + Cross Entropy
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_model = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_model, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

## Running The Transfer Learning Session

In [ ]:
# Launch the graph
with tf.Session() as sess:
    
    #First let's load meta graph and restore weights from previous LeNet training of MNIST Dataset
    saver = tf.train.import_meta_graph('./LeNet-Model.meta')
    saver.restore(sess,tf.train.latest_checkpoint('./'))
    
    #Initialize variables
    sess.run(init)
    
    #Assign the retrieved values to the variables
    graph = tf.get_default_graph()
    wc1 = graph.get_tensor_by_name('wc1:0')
    wc2 = graph.get_tensor_by_name('wc2:0')
    wd1 = graph.get_tensor_by_name('wd1:0')
    weight_out = graph.get_tensor_by_name('weight_out:0')

    bc1 = graph.get_tensor_by_name('bc1:0')
    bc2 = graph.get_tensor_by_name('bc2:0')
    bd1 = graph.get_tensor_by_name('bd1:0')
    bias_out = graph.get_tensor_by_name('bias_out:0')
    
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = fashion.train.next_batch(batch_size)   
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout}) 
        if step % display_step == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x, y: batch_y, keep_prob: 1.})
            print("Iter " + str(step*batch_size) + ", Loss= " + \
                  "{:.3f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))
        
        step += 1
        
    
    # Calculate accuracy for 256 fashion mnist test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: fashion.test.images[:256],
                                      y: fashion.test.labels[:256],
                                      keep_prob: 1.}))